In [1]:
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import folium
import time
import math
from IPython.display import display
%matplotlib inline
matplotlib.style.use('ggplot')
import lasagne

/Users/XueqiWang/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/XueqiWang/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


### 1.  date

In [18]:
a = '24/7/2016'
a = time.strptime(a,"%d/%m/%Y")
type(a)
print time.strftime('%w',a) # Weekday as a decimal number, where 0 is Sunday and 6 is Saturday.
print time.strftime('%W',a) # Week number of the year (Monday as the first day of the week) 
                            # as a decimal number [00,53]. 
                            # All days in a new year preceding the first Monday are considered to 
                            # be in week 0.


0
29


### 2. Experiment on 2014 dataset

2.1 Encoding

In [72]:
import math
def one_hot_code(encode, column, start, npdata):
    rawdata = df[column].values
    i = 0
    for j in rawdata:
        npdata[i, start + encode[j]] = 1
        i+=1


df = pd.read_csv('data2/Accidents_kis_2014.csv')
## select features
df = df[['Number_of_Vehicles', 'Date', 'Time','Longitude', 
         'Latitude', 'Speed_limit', 'Junction_Detail', 'Junction_Control', 'Light_Conditions', 
         'Weather_Conditions','Road_Surface_Conditions', 'Special_Conditions_at_Site', 
         'Urban_or_Rural_Area', 'Accident_Severity','Number_of_KIS']]

## drop missing data
# weather_Conditions
# Road_Surface_Conditions
df = df.drop(df[df.Road_Surface_Conditions < 0].index)
# Special_Conditions_at_site
# Urban_or_Rural_Area

## Create numpy array
npdata = np.zeros(shape=(len(df),57))  # features(0-52)|Accident_severity(53-55)|number_of_kis

## number of vehicles
npdata[:,0] = df['Number_of_Vehicles'].values

## month, week, weekday
day = df['Date'].values
i = 0
for d in day:
    d = time.strptime(d, '%d/%m/%Y')
    npdata[i,1] = time.strftime('%m', d) # month
    npdata[i,2] = time.strftime('%W', d) # week
    npdata[i,3] = time.strftime('%w', d) # weekday
    i += 1
    
## time1 and time2 (sin and cos)
tt = df['Time'].values
i = 0
for t in tt:
    hm = t.split(':')
    t = float(hm[0])*60 + float(hm[1])
    npdata[i,4] = math.sin(2* math.pi * t/1440)
    npdata[i,5] = math.cos(2* math.pi * t/1440)
    
## longitude and latitude
npdata[:,6] = df['Longitude'].values
npdata[:,7] = df['Latitude'].values

## speed limit
npdata[:,8] = df['Speed_limit'].values

## Junction Detail
encode = {0: 0, 1:1, 2: 2, 3:3, 5:4, 6:5, 7:6, 8:7, 9:8}
one_hot_code(encode = encode, column = 'Junction_Detail', start = 9, npdata = npdata)

## Junction Control
junc = df['Junction_Control'].values
i = 0
for j in junc:
    if j == -1:
        npdata[i,18] = 1
    else:
        npdata[i, 18+j] = 1
    i+=1

## Light Conditions
encode = {1:0, 4:1, 5:2, 6:3, 7:4}
one_hot_code(encode = encode, column = 'Light_Conditions', start = 23, npdata = npdata)

## Weather Conditions
encode = {1:0, 2: 1, 3: 2, 4: 3, 5:4, 6:5, 7:6, 8:7, 9:8}
one_hot_code(encode = encode, column = 'Weather_Conditions', start = 28, npdata = npdata)

## Road surface conditions
encode = {1:0, 2: 1, 3: 2, 4: 3, 5:4, 6:5, 7:6}
one_hot_code(encode = encode, column = 'Road_Surface_Conditions', start = 37, npdata = npdata)

## Special conditions at site
encode = {0:0, 1: 1, 2: 2, 3: 3, 4:4, 5:5, 6:6, 7:7}
one_hot_code(encode = encode, column = 'Special_Conditions_at_Site', start = 44, npdata = npdata)

## urban or rural
ur = df['Urban_or_Rural_Area'].values
i = 0
for j in ur:
    if j==1:
        npdata[i, 52] = 1
    else:
        npdata[i, 52] = -1
    i+=1

## Accident severity
encode = {1:0, 2:1, 3:2}
one_hot_code(encode = encode, column = 'Accident_Severity', start = 53, npdata = npdata)

## Number of KIS
npdata[:,56] = df['Number_of_KIS'].values

np.save('data2/demo_2014_numpy_encoded.npy', npdata) # implement encoding, wait for normalisation

2.2 Normalization

In [2]:
npdata = np.load('data2/demo_2014_numpy_encoded.npy')
clm = [0, 1, 2, 3,6,7,8]
for c in clm:
    norm = npdata[:,c]
    norm = (norm - np.mean(norm))/np.std(norm)
    npdata[:,c] = norm

np.save('data2/demo_2014_numpy_standardised.npy', npdata)

2.3 Neural network trail - 2014 dataset

In [5]:
np.shape(npdata)

(5625, 57)

In [19]:
from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne

def load_dataset():
    npdata = np.load('data2/demo_2014_numpy_standardised.npy')
#     (m,n) = np.shape(npdata)
#     npdata = npdata.reshape((m,1,n))
    npdata = npdata.astype(np.float32)
    X_train, X_val = npdata[0:4000, 0:53].reshape(4000,1,53), npdata[4000:4800, 0:53].reshape(800,1,53)
    y_train, y_val = npdata[0:4000, 56].reshape(4000,1), npdata[4000:4800, 56].reshape(800,1)
    x_test, y_test = npdata[4800:5625, 0:53].reshape(825,1,53), npdata[4800:5625, 56].reshape(825,1)
    #print (X_train.shape, y_train.shape, X_val.shape, y_val.shape, npdata.shape)
    return X_train, y_train, X_val, y_val, x_test, y_test


# ##################### Build the neural network model #######################
# This script supports three types of models. For each one, we define a
# function that takes a Theano variable representing the input and returns
# the output layer of a neural network model built in Lasagne.

def build_mlp(input_var=None):
    # This creates an MLP of two hidden layers of 800 units each, followed by
    # a softmax output layer of 10 units. It applies 20% dropout to the input
    # data and 50% dropout to the hidden layers.

    # Input layer, specifying the expected input shape of the network
    # (unspecified batchsize, 1 channel, 28 rows and 28 columns) and
    # linking it to the given Theano variable `input_var`, if any:
    l_in = lasagne.layers.InputLayer(shape=(None,1,53),
                                     input_var=input_var)

    # Apply 20% dropout to the input data:
    #l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)

    # Add a fully-connected layer of 800 units, using the linear rectifier, and
    # initializing weights with Glorot's scheme (which is the default anyway):
    l_hid1 = lasagne.layers.DenseLayer(
            l_in, num_units=100,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    # We'll now add dropout of 50%:
    #l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)

    # Another 800-unit layer:
    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1, num_units=100,
            nonlinearity=lasagne.nonlinearities.rectify)

    # 50% dropout again:
    #l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.5)

    # Finally, we'll add the fully-connected output layer, of 10 softmax units:
    l_out = lasagne.layers.DenseLayer(
            l_hid2, num_units=1,
            nonlinearity=lasagne.nonlinearities.linear)

    # Each layer is linked to its incoming layer(s), so we only need to pass
    # the output layer to give access to a network in Lasagne:
    return l_out


# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.
# Notice that this function returns only mini-batches of size `batchsize`.
# If the size of the data is not a multiple of `batchsize`, it will not
# return the last (remaining) mini-batch.

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]


# ############################## Main program ################################
# Everything else will be handled in our main program now. We could pull out
# more functions to better separate the code, but it wouldn't make it any
# easier to read.

def main(model='mlp', num_epochs=500):
    # Load the dataset
    print("Loading data...")
    X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

    # Prepare Theano variables for inputs and targets
    input_var = T.tensor3('inputs')
    target_var = T.matrix('targets')

    # Create neural network model (depending on first command line parameter)
    print("Building model and compiling functions...")
    if model == 'mlp':
        network = build_mlp(input_var)
    elif model.startswith('custom_mlp:'):
        depth, width, drop_in, drop_hid = model.split(':', 1)[1].split(',')
        network = build_custom_mlp(input_var, int(depth), int(width),
                                   float(drop_in), float(drop_hid))
    elif model == 'cnn':
        network = build_cnn(input_var)
    else:
        print("Unrecognized model type %r." % model)
        return

    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    prediction = lasagne.layers.get_output(network)
    loss = lasagne.objectives.squared_error(prediction, target_var)
    loss = loss.mean()
    # We could add some weight decay as well here, see lasagne.regularization.

    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step. Here, we'll use Stochastic Gradient
    # Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
    params = lasagne.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.01, momentum=0.9)

    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout layers.
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.squared_error(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    # As a bonus, also create an expression for the classification accuracy:
    # no accuracy for regression

    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_fn = theano.function([input_var, target_var], loss, updates=updates, allow_input_downcast = True)

    # Compile a second function computing the validation loss and accuracy:
    val_fn = theano.function([input_var, target_var], test_loss, allow_input_downcast = True)

    # Finally, launch the training loop.
    print("Starting training...")
    # We iterate over epochs:
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, 10, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # And a full pass over the validation data:
        val_err = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, y_val.shape[0], shuffle=False):
            inputs, targets = batch
            #print (inputs.shape, targets.shape)
            err = val_fn(inputs, targets)
            val_err += err
            val_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        

    # After training, we compute and print the test error:
    test_err = 0
    test_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
        inputs, targets = batch
        err = val_fn(inputs, targets)
        test_err += err
        test_batches += 1
    print("Final results:")
    print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))

    # Optionally, you could now dump the network weights to a file like this:
    # np.savez('model.npz', *lasagne.layers.get_all_param_values(network))
    #
    # And load them again later on like this:
    # with np.load('model.npz') as f:
    #     param_values = [f['arr_%d' % i] for i in range(len(f.files))]
    # lasagne.layers.set_all_param_values(network, param_values)


if __name__ == '__main__':
    if ('--help' in sys.argv) or ('-h' in sys.argv):
        print("Trains a neural network on MNIST using Lasagne.")
        print("Usage: %s [MODEL [EPOCHS]]" % sys.argv[0])
        print()
        print("MODEL: 'mlp' for a simple Multi-Layer Perceptron (MLP),")
        print("       'custom_mlp:DEPTH,WIDTH,DROP_IN,DROP_HID' for an MLP")
        print("       with DEPTH hidden layers of WIDTH units, DROP_IN")
        print("       input dropout and DROP_HID hidden dropout,")
        print("       'cnn' for a simple Convolutional Neural Network (CNN).")
        print("EPOCHS: number of training epochs to perform (default: 500)")
    else:
        kwargs = {}
        if len(sys.argv) > 1:
            kwargs['model'] = 'mlp'
        if len(sys.argv) > 2:
            kwargs['num_epochs'] = 100
        main(**kwargs)

Loading data...
Building model and compiling functions...
Starting training...
Epoch 1 of 100 took 0.092s
  training loss:		0.191360
  validation loss:		0.241574
Epoch 2 of 100 took 0.091s
  training loss:		0.179982
  validation loss:		0.239769
Epoch 3 of 100 took 0.106s
  training loss:		0.177733
  validation loss:		0.237417
Epoch 4 of 100 took 0.093s
  training loss:		0.173950
  validation loss:		0.237353
Epoch 5 of 100 took 0.091s
  training loss:		0.172813
  validation loss:		0.243403
Epoch 6 of 100 took 0.100s
  training loss:		0.169708
  validation loss:		0.242203
Epoch 7 of 100 took 0.091s
  training loss:		0.168837
  validation loss:		0.241939
Epoch 8 of 100 took 0.091s
  training loss:		0.166193
  validation loss:		0.254769
Epoch 9 of 100 took 0.103s
  training loss:		0.165298
  validation loss:		0.242451
Epoch 10 of 100 took 0.092s
  training loss:		0.162526
  validation loss:		0.250024
Epoch 11 of 100 took 0.092s
  training loss:		0.160275
  validation loss:		0.243868
Epoch 

## Data standardisation - the rest of the years

1. Data cleaning - remove missing data in each year
2. Data encoding

In [75]:
import math
def one_hot_code(encode, column, start, npdata):
    rawdata = df[column].values
    i = 0
    for j in rawdata:
        npdata[i, start + encode[j]] = 1
        i+=1

years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015']
for year in years:
    df = pd.read_csv('data2/Accidents_kis_' + year + '.csv')
    ## select features
    df = df[['Number_of_Vehicles', 'Date', 'Time','Longitude', 
             'Latitude', 'Speed_limit', 'Junction_Detail', 'Junction_Control', 'Light_Conditions', 
             'Weather_Conditions','Road_Surface_Conditions', 'Special_Conditions_at_Site', 
             'Urban_or_Rural_Area', 'Accident_Severity','Number_of_KIS']]

    ## drop missing data
    # weather_Conditions
    df = df.drop(df[df.Weather_Conditions < 0].index)
    # Road_Surface_Conditions
    df = df.drop(df[df.Road_Surface_Conditions < 0].index)
    # Special_Conditions_at_site
    df = df.drop(df[df.Special_Conditions_at_Site < 0].index)
    # Urban_or_Rural_Area
    df = df.drop(df[df.Urban_or_Rural_Area < 0].index)
    df = df.dropna()
    
    print ('The number of samples in '+year+ ': ', len(df))
    ## Create numpy array
    npdata = np.zeros(shape=(len(df),57))  # features(0-52)|Accident_severity(53-55)|number_of_kis

    ## number of vehicles
    npdata[:,0] = df['Number_of_Vehicles'].values

    ## month, week, weekday
    day = df['Date'].values
    i = 0
    for d in day:
        d = time.strptime(d, '%d/%m/%Y')
        npdata[i,1] = time.strftime('%m', d) # month
        npdata[i,2] = time.strftime('%W', d) # week
        npdata[i,3] = time.strftime('%w', d) # weekday
        i += 1

    ## time1 and time2 (sin and cos)
    tt = df['Time'].values
    i = 0
    for t in tt:
        hm = t.split(':')
        t = float(hm[0])*60 + float(hm[1])
        npdata[i,4] = math.sin(2* math.pi * t/1440)
        npdata[i,5] = math.cos(2* math.pi * t/1440)

    ## longitude and latitude
    npdata[:,6] = df['Longitude'].values
    npdata[:,7] = df['Latitude'].values

    ## speed limit
    npdata[:,8] = df['Speed_limit'].values

    ## Junction Detail
    encode = {0: 0, 1:1, 2: 2, 3:3, 5:4, 6:5, 7:6, 8:7, 9:8}
    one_hot_code(encode = encode, column = 'Junction_Detail', start = 9, npdata = npdata)

    ## Junction Control
    junc = df['Junction_Control'].values
    i = 0
    for j in junc:
        if j == -1:
            npdata[i,18] = 1
        else:
            npdata[i, 18+j] = 1
        i+=1

    ## Light Conditions
    encode = {1:0, 4:1, 5:2, 6:3, 7:4}
    one_hot_code(encode = encode, column = 'Light_Conditions', start = 23, npdata = npdata)

    ## Weather Conditions
    encode = {1:0, 2: 1, 3: 2, 4: 3, 5:4, 6:5, 7:6, 8:7, 9:8}
    one_hot_code(encode = encode, column = 'Weather_Conditions', start = 28, npdata = npdata)

    ## Road surface conditions
    encode = {1:0, 2: 1, 3: 2, 4: 3, 5:4, 6:5, 7:6}
    one_hot_code(encode = encode, column = 'Road_Surface_Conditions', start = 37, npdata = npdata)

    ## Special conditions at site
    encode = {0:0, 1: 1, 2: 2, 3: 3, 4:4, 5:5, 6:6, 7:7}
    one_hot_code(encode = encode, column = 'Special_Conditions_at_Site', start = 44, npdata = npdata)

    ## urban or rural
    ur = df['Urban_or_Rural_Area'].values
    i = 0
    for j in ur:
        if j==1:
            npdata[i, 52] = 1
        else:
            npdata[i, 52] = -1
        i+=1

    ## Accident severity
    encode = {1:0, 2:1, 3:2}
    one_hot_code(encode = encode, column = 'Accident_Severity', start = 53, npdata = npdata)

    ## Number of KIS
    npdata[:,56] = df['Number_of_KIS'].values

    np.save('data3/Accidents_'+ year + '_numpy_encoded.npy', npdata) # save each year's data as numpy array
    #implement encoding, wait for normalisation

The number of samples in 2009:  6638
The number of samples in 2010:  6499
The number of samples in 2011:  5819
The number of samples in 2012:  5607
The number of samples in 2013:  5388
The number of samples in 2014:  5625
The number of samples in 2015:  5537


Data normalisation

In [77]:
npdata = np.load('data3/Accidents_2009_numpy_encoded.npy') # initialise numpy array
print ('2009:', npdata.shape)
years = ['2010', '2011', '2012', '2013', '2014', '2015']
for year in years:
    newdata = np.load('data3/Accidents_' + year + '_numpy_encoded.npy')
    print (year+':', newdata.shape)
    npdata = np.vstack([npdata,newdata])
    
print ('all data:', npdata.shape)

clm = [0, 1, 2, 3,6,7,8]
for c in clm:
    norm = npdata[:,c]
    norm = (norm - np.mean(norm))/np.std(norm)
    npdata[:,c] = norm

np.save('data3/Accidents_allyear_numpy_standardised.npy', npdata)

2009: (6638, 57)
2010: (6499, 57)
2011: (5819, 57)
2012: (5607, 57)
2013: (5388, 57)
2014: (5625, 57)
2015: (5537, 57)
all data: (41113, 57)


## Neural Network - 1
- predict the number of KIS
- training data: 2009-2013
- validaton data: 2014
- test data: 2015

In [131]:
from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne

def load_dataset():
    npdata = np.load('data3/Accidents_allyear_numpy_standardised.npy')
    npdata = npdata.astype(np.float32)
    X_train, X_val = npdata[0:29951, 0:53].reshape(29951,1,53), npdata[29951:35576, 0:53].reshape(5625,1,53)
    y_train, y_val = npdata[0:29951, 56].reshape(29951,1), npdata[29951:35576, 56].reshape(5625,1)
    x_test, y_test = npdata[35576:41113, 0:53].reshape(5537,1,53), npdata[35576:41113, 56].reshape(5537,1 )
    #print (X_train.shape, y_train.shape, X_val.shape, y_val.shape, npdata.shape)
    return X_train, y_train, X_val, y_val, x_test, y_test


def build_mlp(input_var=None):
    # This creates an MLP of two hidden layers of 800 units each, followed by
    
    # Input layer, specifying the expected input shape of the network
    l_in = lasagne.layers.InputLayer(shape=(None,1,53),
                                     input_var=input_var)

    # Apply 20% dropout to the input data:
    #l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)

    # Add a fully-connected layer of 800 units, using the linear rectifier, and
    # initializing weights with Glorot's scheme (which is the default anyway):
    l_hid1 = lasagne.layers.DenseLayer(
            l_in, num_units=300,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    # We'll now add dropout of 50%:
    #l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)

    # Another 800-unit layer:
    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1, num_units=400,
            nonlinearity=lasagne.nonlinearities.rectify)
    
    l_hid3 = lasagne.layers.DenseLayer(
            l_hid2, num_units=500,
            nonlinearity=lasagne.nonlinearities.rectify)

    # 50% dropout again:
    #l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.5)

    # Finally, we'll add the fully-connected output layer, of 10 softmax units:
    l_out = lasagne.layers.DenseLayer(
            l_hid3, num_units=1,
            nonlinearity=lasagne.nonlinearities.linear)

    # Each layer is linked to its incoming layer(s), so we only need to pass
    # the output layer to give access to a network in Lasagne:
    return l_out


# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.
# Notice that this function returns only mini-batches of size `batchsize`.
# If the size of the data is not a multiple of `batchsize`, it will not
# return the last (remaining) mini-batch.

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]


# ############################## Main program ################################
# Everything else will be handled in our main program now. We could pull out
# more functions to better separate the code, but it wouldn't make it any
# easier to read.

def main(model='mlp', num_epochs=500):
    # Load the dataset
    print("Loading data...")
    X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

    # Prepare Theano variables for inputs and targets
    input_var = T.tensor3('inputs')
    target_var = T.matrix('targets')

    # Create neural network model (depending on first command line parameter)
    print("Building model and compiling functions...")
    if model == 'mlp':
        network = build_mlp(input_var)
    elif model.startswith('custom_mlp:'):
        depth, width, drop_in, drop_hid = model.split(':', 1)[1].split(',')
        network = build_custom_mlp(input_var, int(depth), int(width),
                                   float(drop_in), float(drop_hid))
    elif model == 'cnn':
        network = build_cnn(input_var)
    else:
        print("Unrecognized model type %r." % model)
        return

    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    prediction = lasagne.layers.get_output(network)
    loss = lasagne.objectives.squared_error(prediction, target_var)
    loss = loss.mean()
    # We could add some weight decay as well here, see lasagne.regularization.

    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step. Here, we'll use Stochastic Gradient
    # Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
    params = lasagne.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.0001, momentum=0.9)

    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout layers.
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.squared_error(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    # As a bonus, also create an expression for the classification accuracy:
    # no accuracy for regression

    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_fn = theano.function([input_var, target_var], loss, updates=updates, allow_input_downcast = True)

    # Compile a second function computing the validation loss and accuracy:
    val_fn = theano.function([input_var, target_var], test_loss, allow_input_downcast = True)

    # Finally, launch the training loop.
    print("Starting training...")
    # We iterate over epochs:
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, 10, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # And a full pass over the validation data:
        val_err = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, y_val.shape[0], shuffle=False):
            inputs, targets = batch
            #print (inputs.shape, targets.shape)
            err = val_fn(inputs, targets)
            val_err += err
            val_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        

    # After training, we compute and print the test error:
    test_err = 0
    test_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
        inputs, targets = batch
        err = val_fn(inputs, targets)
        test_err += err
        test_batches += 1
    print("Final results:")
    print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))

    # Optionally, you could now dump the network weights to a file like this:
    # np.savez('model.npz', *lasagne.layers.get_all_param_values(network))
    #
    # And load them again later on like this:
    # with np.load('model.npz') as f:
    #     param_values = [f['arr_%d' % i] for i in range(len(f.files))]
    # lasagne.layers.set_all_param_values(network, param_values)


if __name__ == '__main__':
    if ('--help' in sys.argv) or ('-h' in sys.argv):
        print("Trains a neural network on MNIST using Lasagne.")
        print("Usage: %s [MODEL [EPOCHS]]" % sys.argv[0])
        print()
        print("MODEL: 'mlp' for a simple Multi-Layer Perceptron (MLP),")
        print("       'custom_mlp:DEPTH,WIDTH,DROP_IN,DROP_HID' for an MLP")
        print("       with DEPTH hidden layers of WIDTH units, DROP_IN")
        print("       input dropout and DROP_HID hidden dropout,")
        print("       'cnn' for a simple Convolutional Neural Network (CNN).")
        print("EPOCHS: number of training epochs to perform (default: 500)")
    else:
        kwargs = {}
        if len(sys.argv) > 1:
            kwargs['model'] = 'mlp'
        if len(sys.argv) > 2:
            kwargs['num_epochs'] = 100
        main(**kwargs)

Loading data...
Building model and compiling functions...
Starting training...
Epoch 1 of 100 took 11.231s
  training loss:		0.212666
  validation loss:		0.192709
Epoch 2 of 100 took 12.162s
  training loss:		0.209046
  validation loss:		0.191361
Epoch 3 of 100 took 10.831s
  training loss:		0.207228
  validation loss:		0.191127
Epoch 4 of 100 took 10.953s
  training loss:		0.206416
  validation loss:		0.190783
Epoch 5 of 100 took 13.720s
  training loss:		0.205659
  validation loss:		0.191697
Epoch 6 of 100 took 12.095s
  training loss:		0.205206
  validation loss:		0.191024
Epoch 7 of 100 took 12.112s
  training loss:		0.204852
  validation loss:		0.190848
Epoch 8 of 100 took 13.609s
  training loss:		0.204485
  validation loss:		0.190995
Epoch 9 of 100 took 13.351s
  training loss:		0.204161
  validation loss:		0.190870
Epoch 10 of 100 took 12.117s
  training loss:		0.203737
  validation loss:		0.191027
Epoch 11 of 100 took 12.087s
  training loss:		0.203432
  validation loss:		0.19

KeyboardInterrupt: 

not encode label of accident severity

In [91]:
import math
def one_hot_code(encode, column, start, npdata):
    rawdata = df[column].values
    i = 0
    for j in rawdata:
        npdata[i, start + encode[j]] = 1
        i+=1

years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015']
for year in years:
    df = pd.read_csv('data2/Accidents_kis_' + year + '.csv')
    ## select features
    df = df[['Number_of_Vehicles', 'Date', 'Time','Longitude', 
             'Latitude', 'Speed_limit', 'Junction_Detail', 'Junction_Control', 'Light_Conditions', 
             'Weather_Conditions','Road_Surface_Conditions', 'Special_Conditions_at_Site', 
             'Urban_or_Rural_Area', 'Accident_Severity','Number_of_KIS']]

    ## drop missing data
    # weather_Conditions
    df = df.drop(df[df.Weather_Conditions < 0].index)
    # Road_Surface_Conditions
    df = df.drop(df[df.Road_Surface_Conditions < 0].index)
    # Special_Conditions_at_site
    df = df.drop(df[df.Special_Conditions_at_Site < 0].index)
    # Urban_or_Rural_Area
    df = df.drop(df[df.Urban_or_Rural_Area < 0].index)
    df = df.dropna()
    
    print ('The number of samples in '+year+ ': ', len(df))
    ## Create numpy array
    npdata = np.zeros(shape=(len(df),55))  # features(0-52)|Accident_severity(53-55)|number_of_kis

    ## number of vehicles
    npdata[:,0] = df['Number_of_Vehicles'].values

    ## month, week, weekday
    day = df['Date'].values
    i = 0
    for d in day:
        d = time.strptime(d, '%d/%m/%Y')
        npdata[i,1] = time.strftime('%m', d) # month
        npdata[i,2] = time.strftime('%W', d) # week
        npdata[i,3] = time.strftime('%w', d) # weekday
        i += 1

    ## time1 and time2 (sin and cos)
    tt = df['Time'].values
    i = 0
    for t in tt:
        hm = t.split(':')
        t = float(hm[0])*60 + float(hm[1])
        npdata[i,4] = math.sin(2* math.pi * t/1440)
        npdata[i,5] = math.cos(2* math.pi * t/1440)

    ## longitude and latitude
    npdata[:,6] = df['Longitude'].values
    npdata[:,7] = df['Latitude'].values

    ## speed limit
    npdata[:,8] = df['Speed_limit'].values

    ## Junction Detail
    encode = {0: 0, 1:1, 2: 2, 3:3, 5:4, 6:5, 7:6, 8:7, 9:8}
    one_hot_code(encode = encode, column = 'Junction_Detail', start = 9, npdata = npdata)

    ## Junction Control
    junc = df['Junction_Control'].values
    i = 0
    for j in junc:
        if j == -1:
            npdata[i,18] = 1
        else:
            npdata[i, 18+j] = 1
        i+=1

    ## Light Conditions
    encode = {1:0, 4:1, 5:2, 6:3, 7:4}
    one_hot_code(encode = encode, column = 'Light_Conditions', start = 23, npdata = npdata)

    ## Weather Conditions
    encode = {1:0, 2: 1, 3: 2, 4: 3, 5:4, 6:5, 7:6, 8:7, 9:8}
    one_hot_code(encode = encode, column = 'Weather_Conditions', start = 28, npdata = npdata)

    ## Road surface conditions
    encode = {1:0, 2: 1, 3: 2, 4: 3, 5:4, 6:5, 7:6}
    one_hot_code(encode = encode, column = 'Road_Surface_Conditions', start = 37, npdata = npdata)

    ## Special conditions at site
    encode = {0:0, 1: 1, 2: 2, 3: 3, 4:4, 5:5, 6:6, 7:7}
    one_hot_code(encode = encode, column = 'Special_Conditions_at_Site', start = 44, npdata = npdata)

    ## urban or rural
    ur = df['Urban_or_Rural_Area'].values
    i = 0
    for j in ur:
        if j==1:
            npdata[i, 52] = 1
        else:
            npdata[i, 52] = -1
        i+=1

    ## Accident severity, do not encode
    npdata[:,53] = df['Accident_Severity'].values

    ## Number of KIS
    npdata[:,54] = df['Number_of_KIS'].values

    np.save('data3/Accidents_'+ year + '_numpy_encoded2.npy', npdata) # save each year's data as numpy array
    #implement encoding, wait for normalisation

The number of samples in 2009:  6638
The number of samples in 2010:  6499
The number of samples in 2011:  5819
The number of samples in 2012:  5607
The number of samples in 2013:  5388
The number of samples in 2014:  5625
The number of samples in 2015:  5537


In [93]:
npdata = np.load('data3/Accidents_2009_numpy_encoded2.npy') # initialise numpy array
print ('2009:', npdata.shape)
years = ['2010', '2011', '2012', '2013', '2014', '2015']
for year in years:
    newdata = np.load('data3/Accidents_' + year + '_numpy_encoded2.npy')
    print (year+':', newdata.shape)
    npdata = np.vstack([npdata,newdata])
    
print ('all data:', npdata.shape)

clm = [0, 1, 2, 3,6,7,8]
for c in clm:
    norm = npdata[:,c]
    norm = (norm - np.mean(norm))/np.std(norm)
    npdata[:,c] = norm

np.save('data3/Accidents_allyear_numpy_standardised2.npy', npdata)

2009: (6638, 55)
2010: (6499, 55)
2011: (5819, 55)
2012: (5607, 55)
2013: (5388, 55)
2014: (5625, 55)
2015: (5537, 55)
all data: (41113, 55)


In [103]:
npdata = np.load('data3/Accidents_allyear_numpy_standardised2.npy')
sum(np.isnan(npdata)*1.0)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])

## accident severity classification

In [132]:
from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne


# ################## Download and prepare the MNIST dataset ##################
# This is just some way of getting the MNIST dataset from an online location
# and loading it into numpy arrays. It doesn't involve Lasagne at all.

def load_dataset():
    
    npdata = np.load('data3/Accidents_allyear_numpy_standardised2.npy')
    npdata = npdata.astype(np.float32)
    X_train, X_val = npdata[0:29951, 0:53].reshape(29951,1,53), npdata[29951:35576, 0:53].reshape(5625,1,53)
    y_train, y_val = npdata[0:29951, 53]-1, npdata[29951:35576, 53]-1
    x_test, y_test = npdata[35576:41113, 0:53].reshape(5537,1,53), npdata[35576:41113, 53]-1
    print (X_train.shape, y_train.shape, X_val.shape, y_val.shape, npdata.shape)
    return X_train, y_train, X_val, y_val, x_test, y_test


# ##################### Build the neural network model #######################
# This script supports three types of models. For each one, we define a
# function that takes a Theano variable representing the input and returns
# the output layer of a neural network model built in Lasagne.

def build_mlp(input_var=None):
    # This creates an MLP of two hidden layers of 800 units each, followed by
    # a softmax output layer of 10 units. It applies 20% dropout to the input
    # data and 50% dropout to the hidden layers.

    # Input layer, specifying the expected input shape of the network
    # (unspecified batchsize, 1 channel, 28 rows and 28 columns) and
    # linking it to the given Theano variable `input_var`, if any:
    l_in = lasagne.layers.InputLayer(shape=(None,1,53),
                                     input_var=input_var)

    # Apply 20% dropout to the input data:
    #l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)

    # Add a fully-connected layer of 800 units, using the linear rectifier, and
    # initializing weights with Glorot's scheme (which is the default anyway):
    l_hid1 = lasagne.layers.DenseLayer(
            l_in, num_units=300,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())

    # We'll now add dropout of 50%:
    #l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)

    # Another 800-unit layer:
    l_hid2 = lasagne.layers.DenseLayer(
            l_hid1, num_units=400,
            nonlinearity=lasagne.nonlinearities.rectify)
    
#     l_hid3 = lasagne.layers.DenseLayer(
#             l_hid2, num_units=500,
#             nonlinearity=lasagne.nonlinearities.rectify)
    
    # Finally, we'll add the fully-connected output layer, of 10 softmax units:
    l_out = lasagne.layers.DenseLayer(
            l_hid2, num_units=3,
            nonlinearity=lasagne.nonlinearities.softmax)

    # Each layer is linked to its incoming layer(s), so we only need to pass
    # the output layer to give access to a network in Lasagne:
    return l_out


def build_custom_mlp(input_var=None, depth=2, width=800, drop_input=.2,
                     drop_hidden=.5):
    # By default, this creates the same network as `build_mlp`, but it can be
    # customized with respect to the number and size of hidden layers. This
    # mostly showcases how creating a network in Python code can be a lot more
    # flexible than a configuration file. Note that to make the code easier,
    # all the layers are just called `network` -- there is no need to give them
    # different names if all we return is the last one we created anyway; we
    # just used different names above for clarity.

    # Input layer and dropout (with shortcut `dropout` for `DropoutLayer`):
    network = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                        input_var=input_var)
    if drop_input:
        network = lasagne.layers.dropout(network, p=drop_input)
    # Hidden layers and dropout:
    nonlin = lasagne.nonlinearities.rectify
    for _ in range(depth):
        network = lasagne.layers.DenseLayer(
                network, width, nonlinearity=nonlin)
        if drop_hidden:
            network = lasagne.layers.dropout(network, p=drop_hidden)
    # Output layer:
    softmax = lasagne.nonlinearities.softmax
    network = lasagne.layers.DenseLayer(network, 10, nonlinearity=softmax)
    return network


def build_cnn(input_var=None):
    # As a third model, we'll create a CNN of two convolution + pooling stages
    # and a fully-connected hidden layer in front of the output layer.

    # Input layer, as usual:
    network = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                        input_var=input_var)
    # This time we do not apply input dropout, as it tends to work less well
    # for convolutional layers.

    # Convolutional layer with 32 kernels of size 5x5. Strided and padded
    # convolutions are supported as well; see the docstring.
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
    # Expert note: Lasagne provides alternative convolutional layers that
    # override Theano's choice of which implementation to use; for details
    # please see http://lasagne.readthedocs.org/en/latest/user/tutorial.html.

    # Max-pooling layer of factor 2 in both dimensions:
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # Another convolution with 32 5x5 kernels, and another 2x2 pooling:
    network = lasagne.layers.Conv2DLayer(
            network, num_filters=32, filter_size=(5, 5),
            nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.MaxPool2DLayer(network, pool_size=(2, 2))

    # A fully-connected layer of 256 units with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=256,
            nonlinearity=lasagne.nonlinearities.rectify)

    # And, finally, the 10-unit output layer with 50% dropout on its inputs:
    network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)

    return network


# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.
# Notice that this function returns only mini-batches of size `batchsize`.
# If the size of the data is not a multiple of `batchsize`, it will not
# return the last (remaining) mini-batch.

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]


# ############################## Main program ################################
# Everything else will be handled in our main program now. We could pull out
# more functions to better separate the code, but it wouldn't make it any
# easier to read.

def main(model='mlp', num_epochs=500):
    # Load the dataset
    print("Loading data...")
    X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

    # Prepare Theano variables for inputs and targets
    input_var = T.tensor3('inputs')
    target_var = T.ivector('targets')

    # Create neural network model (depending on first command line parameter)
    print("Building model and compiling functions...")
    if model == 'mlp':
        network = build_mlp(input_var)
    elif model.startswith('custom_mlp:'):
        depth, width, drop_in, drop_hid = model.split(':', 1)[1].split(',')
        network = build_custom_mlp(input_var, int(depth), int(width),
                                   float(drop_in), float(drop_hid))
    elif model == 'cnn':
        network = build_cnn(input_var)
    else:
        print("Unrecognized model type %r." % model)
        return

    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    prediction = lasagne.layers.get_output(network)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()
    # We could add some weight decay as well here, see lasagne.regularization.

    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step. Here, we'll use Stochastic Gradient
    # Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.
    params = lasagne.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.001, momentum=0.9)

    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout layers.
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    # As a bonus, also create an expression for the classification accuracy:
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_fn = theano.function([input_var, target_var], loss, updates=updates, allow_input_downcast=True)

    # Compile a second function computing the validation loss and accuracy:
    val_fn = theano.function([input_var, target_var], [test_loss, test_acc], allow_input_downcast=True)

    # Finally, launch the training loop.
    print("Starting training...")
    # We iterate over epochs:
    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, 10, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1

        # And a full pass over the validation data:
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, 10, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs, targets)
            val_err += err
            val_acc += acc
            val_batches += 1

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))

    # After training, we compute and print the test error:
    test_err = 0
    test_acc = 0
    test_batches = 0
    for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        test_err += err
        test_acc += acc
        test_batches += 1
    print("Final results:")
    print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
    print("  test accuracy:\t\t{:.2f} %".format(
        test_acc / test_batches * 100))

    # Optionally, you could now dump the network weights to a file like this:
    # np.savez('model.npz', *lasagne.layers.get_all_param_values(network))
    #
    # And load them again later on like this:
    # with np.load('model.npz') as f:
    #     param_values = [f['arr_%d' % i] for i in range(len(f.files))]
    # lasagne.layers.set_all_param_values(network, param_values)


if __name__ == '__main__':
    if ('--help' in sys.argv) or ('-h' in sys.argv):
        print("Trains a neural network on MNIST using Lasagne.")
        print("Usage: %s [MODEL [EPOCHS]]" % sys.argv[0])
        print()
        print("MODEL: 'mlp' for a simple Multi-Layer Perceptron (MLP),")
        print("       'custom_mlp:DEPTH,WIDTH,DROP_IN,DROP_HID' for an MLP")
        print("       with DEPTH hidden layers of WIDTH units, DROP_IN")
        print("       input dropout and DROP_HID hidden dropout,")
        print("       'cnn' for a simple Convolutional Neural Network (CNN).")
        print("EPOCHS: number of training epochs to perform (default: 500)")
    else:
        kwargs = {}
        if len(sys.argv) > 1:
            kwargs['model'] = 'mlp'
        if len(sys.argv) > 2:
            kwargs['num_epochs'] = 50
        main(**kwargs)

Loading data...
(29951, 1, 53) (29951,) (5625, 1, 53) (5625,) (41113, 55)
Building model and compiling functions...
Starting training...
Epoch 1 of 50 took 5.095s
  training loss:		0.415503
  validation loss:		0.405417
  validation accuracy:		87.92 %
Epoch 2 of 50 took 5.269s
  training loss:		0.403370
  validation loss:		0.400768
  validation accuracy:		87.92 %
Epoch 3 of 50 took 5.042s
  training loss:		0.400325
  validation loss:		0.400616
  validation accuracy:		87.94 %
Epoch 4 of 50 took 5.154s
  training loss:		0.397930
  validation loss:		0.397971
  validation accuracy:		87.94 %
Epoch 5 of 50 took 5.200s
  training loss:		0.396233
  validation loss:		0.399154
  validation accuracy:		87.92 %
Epoch 6 of 50 took 5.578s
  training loss:		0.395464
  validation loss:		0.397706
  validation accuracy:		87.94 %
Epoch 7 of 50 took 5.659s
  training loss:		0.394347
  validation loss:		0.397335
  validation accuracy:		87.92 %
Epoch 8 of 50 took 5.593s
  training loss:		0.393513
  validation

In [127]:
data = np.load('data3/Accidents_allyear_numpy_standardised2.npy')
a = pd.Series(data[:,53])

a.value_counts()

3.0    36170
2.0     4264
1.0      679
dtype: int64

In [129]:
36170.0/(36170+4264+679)

0.8797703889280762

In [130]:
data = np.load('data3/Accidents_allyear_numpy_standardised2.npy')
a = pd.Series(data[:,54])
a.value_counts()

0.0     36170
1.0      4307
2.0       466
3.0       102
4.0        37
6.0        11
5.0        11
7.0         4
8.0         2
19.0        1
18.0        1
9.0         1
dtype: int64